# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Небходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.



---


**Цель исследования:** определить регион с наибольшей прибыльностью разработки месторождений нефти.  

**Задачи:**  
- Изучить и подготовить данные;  
- Обучить модели для предсказания объемов запасов нефти в регионах;  
- Определить минимальный объем нефти, необходимый для безубыточности разработки скважины;  
- Оценить прибыль от разработки месторождений с наибольшими предсказанными запасами нефти;  
- Проанализировать техникой *Bootstrap* распределение прибыли и определить риски убытков;  
- Определить регион с наименьшими рисками и наибольшей прибыльностью разработки месторождений нефти.

В нашем распоряжении данные геологоразведки трёх регионов:  
- `id` — уникальный идентификатор скважины;  
- `f0`, `f1`, `f2` — три признака точек;  
- `product` — объём запасов в скважине (тыс. баррелей).

Также, есть ряд условий, которые необходимо учесть:  
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
- Бюджет на разработку скважин в регионе — 10 млрд рублей.  
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [ ]:
!pip install scikit-learn==1.5.2 -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

RANDOM_STATE = 42

## Загрузка и подготовка данных

In [ ]:
try:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('geo_data_0.csv')
    data_1 = pd.read_csv('geo_data_1.csv')
    data_2 = pd.read_csv('geo_data_2.csv')

In [ ]:
display(data_0.head())
display(data_1.head())
display(data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Данные отображаются корректно, передадим столбец `id` в качестве индексов датафреймов

In [ ]:
data_0 = data_0.set_index('id')
data_1 = data_1.set_index('id')
data_2 = data_2.set_index('id')

Определим функцию для вывода основной информации

In [ ]:
def data_info(df):
    display(df.head())
    print()
    df.info()
    print()
    print('Количество явных дубликатов:', df.duplicated().sum())
    print('Количество повторяющихся id:', df.index.duplicated().sum())
    print()
    print('Количество пропущенных значений:\n', df.isna().sum())

### Первый регион

In [ ]:
data_info(data_0)

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647



<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 10

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


Типы данных установленны корректно, пропущенных значений и явных дубликатов нет, но обнаружены неявные среди идентификаторов пользователей. Выведем повторяющиеся `id`:

In [ ]:
for i in data_0[data_0.index.duplicated()].index:
    display(data_0[data_0.index == i])

,f0,f1,f2,product
id,,,,
HZww2,0.755284,0.368511,1.863211,30.681774
HZww2,1.061194,-0.373969,10.430210,158.828695


,f0,f1,f2,product
id,,,,
bxg6G,0.411645,0.856830,-3.653440,73.604260
bxg6G,-0.823752,0.546319,3.630479,93.007798


,f0,f1,f2,product
id,,,,
A5aEY,-0.039949,0.156872,0.209861,89.249364
A5aEY,-0.180335,0.935548,-2.094773,33.020205


,f0,f1,f2,product
id,,,,
QcMuo,0.506563,-0.323775,-2.215583,75.496502
QcMuo,0.635635,-0.473422,0.862670,64.578675


,f0,f1,f2,product
id,,,,
74z30,0.741456,0.459229,5.153109,140.771492
74z30,1.084962,-0.312358,6.990771,127.643327


,f0,f1,f2,product
id,,,,
AGS9W,1.454747,-0.479651,0.683380,126.370504
AGS9W,-0.933795,0.116194,-3.655896,19.230453


,f0,f1,f2,product
id,,,,
Tdehs,0.829407,0.298807,-0.049563,96.035308
Tdehs,0.112079,0.430296,3.218993,60.964018


,f0,f1,f2,product
id,,,,
fiKDv,0.157341,1.028359,5.585586,95.817889
fiKDv,0.049883,0.841313,6.394613,137.346586


,f0,f1,f2,product
id,,,,
TtcGQ,0.569276,-0.104876,6.440215,85.350186
TtcGQ,0.110711,1.022689,0.911381,101.318008


,f0,f1,f2,product
id,,,,
bsk9y,0.398908,-0.400253,10.122376,163.433078
bsk9y,0.378429,0.005837,0.160827,160.637302


у 10 пар наблюдений повторяются идентификаторы, при этом характеристики и значения целевого признака различаются. Так как невозможно установить, какие наблюдения являются корректными, можем исключить их из рассмотрения. Маловероятно, что этот шаг окажет сущетсвенное влияние на качество модели, поскольку таких наблюдений в датасете 20 из 100 000, но всё же сохраним исходный датасет в отдельную переменную.

In [ ]:
data_0_old = data_0

data_0 = data_0.drop(data_0[data_0.index.duplicated()].index)

data_info(data_0)

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647



<class 'pandas.core.frame.DataFrame'>
Index: 99980 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99980 non-null  float64
 1   f1       99980 non-null  float64
 2   f2       99980 non-null  float64
 3   product  99980 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 0

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


### Второй регион

Аналогично проверим и обработаем датасеты по второму и третьему регионам

In [ ]:
data_info(data_1)

,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305



<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 4

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


У 4 пар наблюдений из 100 000 строк `id` не уникален, удалим их, предварительно сохранив исходный датасет.

In [ ]:
data_1_old = data_1

data_1 = data_1.drop(data_1[data_1.index.duplicated()].index)

data_info(data_1)

,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305



<class 'pandas.core.frame.DataFrame'>
Index: 99992 entries, kBEdx to relB0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99992 non-null  float64
 1   f1       99992 non-null  float64
 2   f2       99992 non-null  float64
 3   product  99992 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 0

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


### Третий регион

In [ ]:
data_info(data_2)

,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746



<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 4

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


In [ ]:
data_2_old = data_2

data_2 = data_2.drop(data_2[data_2.index.duplicated()].index)

data_info(data_2)

,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746



<class 'pandas.core.frame.DataFrame'>
Index: 99992 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   f0       99992 non-null  float64
 1   f1       99992 non-null  float64
 2   f2       99992 non-null  float64
 3   product  99992 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB

Количество явных дубликатов: 0
Количество повторяющихся id: 0

Количество пропущенных значений:
 f0         0
f1         0
f2         0
product    0
dtype: int64


### Промежуточный вывод

Данные о скважинах и запасах нефти разделены по регионам на три датасета, каждый из них содержит информацию о трёх характеристиках месторождений (что они означают - не уточняется) и объёме запасов в них. В данных нет пропусков и явных дубликатов, но были обнаружены строки с повторяющимися идентификаторами, они были исключены из дальнейшего рассмотрения.

## Обучение и проверка модели

Обучим модель для каждого региона, для удобства определим функцию, которая будет принимать на вход датафрейм, разделять его на тренировочную и валидационную выборки, масштабировать входные признаки, обучать на полученыых данных модель линейной регрессии и рассчитывать средний объём предсказанного объёма запасов и RMSE модели.

In [ ]:
def lr_model(df):
    X = df.drop(columns='product')
    y = df['product']
    X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                          y,
                                                          test_size=0.25,
                                                          random_state=RANDOM_STATE)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)

    mean_pred = y_pred.mean()
    rmse = root_mean_squared_error(y_valid, y_pred)

    return model, pd.Series(y_valid), pd.Series(y_pred, index=y_valid.index), mean_pred, rmse

Применим функцию и выведем полученные результаты

In [ ]:
model_0, true_0, pred_0, mean_pred_0, rmse_0 = lr_model(data_0)
model_1, true_1, pred_1, mean_pred_1, rmse_1 = lr_model(data_1)
model_2, true_2, pred_2, mean_pred_2, rmse_2 = lr_model(data_2)

for region, mean_pred, rmse in [['Первый регион', mean_pred_0, rmse_0], ['Второй регион', mean_pred_1, rmse_1], ['Третий регион', mean_pred_2, rmse_2]]:
    print(region)
    print('Средний запас предсказанного сырья:', round(mean_pred, 2))
    print('RMSE модели:', round(rmse, 2))
    print()

Первый регион
Средний запас предсказанного сырья: 92.29
RMSE модели: 37.76

Второй регион
Средний запас предсказанного сырья: 69.17
RMSE модели: 0.89

Третий регион
Средний запас предсказанного сырья: 94.82
RMSE модели: 40.13



### Промежуточный вывод

Наибольшие объёмы запасов нефти у скважин в третьем регионе (94,82 тыс. баррелей), далее первый регион (92,29 тыс. баррелей) и затем второй (69,17 тыс. баррелей). При этом лучшее качество у модели, полученной для второго региона, её метрика RMSE наиболее близка к 0 (0,89).

## Подготовка к расчёту прибыли

Обозначим условия для дальнейших расчётов:

In [ ]:
'''

Условия:
При разведке региона исследуют 500 точек (n_samples);
Для разработки выбирают 200 лучших скважин (n_best_wells);
Бюджет на разработку скважин в регионе — 10 млрд рублей (budget);
Доход с каждой единицы продукта - 450 тыс. р. (revenue_per_unit);

'''

N_SAMPLES = 500
N_BEST_WELLS = 200
BUDGET = 10000000000
REVENUE_PER_UNIT = 450000

Определим объём нефти, при котором достигается точка безубыточности

In [ ]:
print('Достаточный объём сырья для безубыточной разработки новой скважины:',
      round((BUDGET / N_BEST_WELLS / REVENUE_PER_UNIT), 2))

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11


Средние запасы предсказанного сырья в каждом регионе ниже того объёма, который необходим для безубыточной разработки месторождений. Оценим распределение прибыли, которую можно получить, если рассматривать 200 лучших скважин с наибольшими предсказанными запасами. Для этого определим функцию для подсчёта прибыли:

In [ ]:
def profit(true, predict, n_best_wells, revenue_per_unit, budget):
    best_predicts = predict.sort_values(ascending=False).head(n_best_wells)
    print('Размер объекта true[best_predicts.index]:', true[best_predicts.index].shape[0])
    profit = true[best_predicts.index].sum() * revenue_per_unit - budget
    return profit

## Расчёт прибыли и рисков

### Bootstrap

Применим технику *Bootstrap* для оценки распределения прибыли:

In [ ]:
def bootstrap(true, predict, n_samples, n_wells, revenue, budget):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        true_subsamples = true.sample(n_samples, random_state=state) # удалила replace=True
        pred_subsamples = predict[true_subsamples.index]

        values.append(profit(true_subsamples, pred_subsamples, n_wells, revenue, budget))

    return pd.Series(values)

In [ ]:
values_0 = bootstrap(true_0, pred_0, N_SAMPLES, N_BEST_WELLS, REVENUE_PER_UNIT, BUDGET)
values_1 = bootstrap(true_1, pred_1, N_SAMPLES, N_BEST_WELLS, REVENUE_PER_UNIT, BUDGET)
values_2 = bootstrap(true_2, pred_2, N_SAMPLES, N_BEST_WELLS, REVENUE_PER_UNIT, BUDGET)

Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.index]: 200
Размер объекта true[best_predicts.

Выведем полученные результаты:

In [ ]:
for region, values in [['Первый регион', values_0], ['Второй регион', values_1], ['Третий регион', values_2]]:
    print(region)
    print(f'Нижняя граница доверительного интервала: {round(values.quantile(0.025)/1000000, 2)} млн.')
    print(f'Верхняя граница доверительного интервала: {round(values.quantile(0.975)/1000000, 2)} млн.')
    print(f'Средняя прибыль: {round(values.mean()/1000000, 2)} млн.')
    print(f'Вероятность убытка: {(values < 0).mean():.1%}')
    print()

Первый регион
Нижняя граница доверительного интервала: -145.56 млн.
Верхняя граница доверительного интервала: 896.06 млн.
Средняя прибыль: 387.84 млн.
Вероятность убытка: 7.5%

Второй регион
Нижняя граница доверительного интервала: 87.77 млн.
Верхняя граница доверительного интервала: 866.77 млн.
Средняя прибыль: 479.73 млн.
Вероятность убытка: 1.2%

Третий регион
Нижняя граница доверительного интервала: -210.1 млн.
Верхняя граница доверительного интервала: 857.43 млн.
Средняя прибыль: 351.41 млн.
Вероятность убытка: 9.5%



### Промежуточный вывод

Наименьший риск убытка у месторождений во втором регионе (1,2%), также там наибольшая средняя прибыль (479,73 млн.р.), а нижняя граница доверительного интервала составляет 87,77 млн.р., при том, что у двух других регионов нижний порог отрицательный. Как по размеру средней прибыли, так и по возможным рискам, оптимальным для разработки месторождений является второй регион.

## Общий вывод

Для проведения исследования была предоставлена информация о трёх характеристиках качества нефти в месторождениях и объёме запасов в них. Данные о скважинах и запасах разделены по регионам на три датасета.  

**Цель исследования:** определить регион с наибольшей прибыльностью разработки месторождений нефти.   

В ходе исследования были выполнены следующие шаги:  
- *изучены и подготовлены данные*  
  устранены наблюдения с повторяющимися идентификаторами, выполнена проверка корректности типов данных, наличия явных
  дубликатов и пропущенных значений;  
- *обучены модели линейной регрессии для каждого региона, выполнена проверка их качества*  
  наилучшей оказалась модель, обученная по данным второго региона;  
- *определён минимальный объём нефти, необходимый для безубыточности разработки скважины*  
  точка безубыточности достигается при объёме запасов нефти более 111,11 тыс. баррелей, при этом ни в одном из рассматриваемых
  регионов средний объём предсказанных запасов не превышает этого значения;  
- *посредством техники Bootstrap проанализировано распределение прибыли для месторождений с наибольшими предсказанными запасами
  нефти*  
  наибольшая средняя прибыль зафиксирована во втором регионе (479,73 млн.р.), как и наиболее высокая нижняя граница доверительного интервала (87,77 млн.р.).
- *оценены риски убытков для каждого региона*  
  наименьший риск убытков также во втором регионе, он составляет 1,2%.
  
Таким образом, оптимальным регионом для разработки месторождений нефти явлется второй регион, как по прибыльности, так и по рискам.